In [7]:
import pandas as pd
import pickle
import keras
from keras.layers import Embedding

In [3]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
PIPA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1rV4t0_UV_wcx--UAHVwkqB8Wa_5n9mnpV05yGG1OHqk/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [22]:
cgl = pd.read_json("../data/large_data/cgl.json")
cgl.head()

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmata,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*,conc_lype
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...",71863,None,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",31,0,6,25,"[[μέγας, κινδυνεύοντας, δέχομαι, ἀείμνηστος, μ..."
6,tlg0006.tlg001.perseus-grc2.xml,Euripides,Cyclops,4141,tlg0006,tlg0006.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...",2535,None,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...","[[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθεν...",7,0,0,1,"[[βοτόν, οὔτις, θύω, θεός, μέγας, γαστήρ, δαίμ..."
7,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,6272,tlg0006,tlg0006.tlg004,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...",3545,None,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...","[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...",11,0,1,2,"[[εἰμί, πολύς, χαίρω, δυσφημέω, ἅζομαι, θέα, σ..."
8,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,8257,tlg0006,tlg0006.tlg005,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...",4898,None,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...","[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...",8,3,8,8,"[[κοιτάζω, λέχος, σός, ναυβάτης, τὶς, πλέω, κρ..."
9,tlg0006.tlg006.perseus-grc2.xml,Euripides,Ἀνδρομάχη,7397,tlg0006,tlg0006.tlg006,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...",4420,None,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...","[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...",5,0,7,4,"[[Τροία, πράσσω, μηδείς, ὅδε, αὐχέω, πράσσω, δ..."


In [4]:
cgl_embeddings = pickle.load(open("../data/large_data/cgl_embeddings.pkl", "rb"))

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [14]:
len(range(1, len(cgl_embeddings[1])+1))

5000

In [18]:
word_index = dict(zip(cgl_embeddings[1],range(1, len(cgl_embeddings[1])+1)))

{'εἰμί': 1,
 'οὗτος': 2,
 'λέγω': 3,
 'ἔχω': 4,
 'γίγνομαι': 5,
 'πολύς': 6,
 'ἄλλος': 7,
 'αὐτός': 8,
 'πᾶς': 9,
 'ποιέω': 10,
 'φημί': 11,
 'λόγος': 12,
 'πόλις': 13,
 'ἀνήρ': 14,
 'οὐδείς': 15,
 'δοκέω': 16,
 'τοιοῦτος': 17,
 'μέγας': 18,
 'δεῖ': 19,
 'οἷος': 20,
 'ἀγαθός': 21,
 'τὶς': 22,
 'σός': 23,
 'ἄνθρωπος': 24,
 'ἕτερος': 25,
 'πρῶτος': 26,
 'λαμβάνω': 27,
 'ἕκαστος': 28,
 'ὁράω': 29,
 'βούλομαι': 30,
 'οἶδα': 31,
 'θεός': 32,
 'ἐκεῖνος': 33,
 'μόνος': 34,
 'χρόνος': 35,
 'σῶμα': 36,
 'ἅπας': 37,
 'νόμος': 38,
 'δίδωμι': 39,
 'φαίνω': 40,
 'ἀρχή': 41,
 'κακός': 42,
 'καλός': 43,
 'φύσις': 44,
 'οἴομαι': 45,
 'ὑπάρχω': 46,
 'παῖς': 47,
 'πρότερος': 48,
 'δύναμαι': 49,
 'χρή': 50,
 'ἀκούω': 51,
 'ἀνάγκη': 52,
 'φίλος': 53,
 'γῆ': 54,
 'μηδείς': 55,
 'δίκαιος': 56,
 'πράσσω': 57,
 'χράω': 58,
 'ὅσος': 59,
 'πατήρ': 60,
 'εἷς': 61,
 'ἔρχομαι': 62,
 'φέρω': 63,
 'πρᾶγμα': 64,
 'γένος': 65,
 'ἡμέρα': 66,
 'ἑαυτοῦ': 67,
 'τρόπος': 68,
 'γυνή': 69,
 'Ἀθηναῖος': 70,
 'ἐμός': 71,
 'ἀλη

In [20]:
# we will generate the concordances again, now focusing on words in the embeddings only
def get_concordances(wordlist, keyword, window, vocabulary=None):
    half = int(window / 2)
    if vocabulary != None:
        wordlist = [w for w in wordlist if w in vocabulary]
    keyword_indices = [el[0] for el in enumerate(wordlist) if el[1]==keyword]
    concordances = [wordlist[i-half:i+half+1] for i in keyword_indices]
    concordances = [c for c in concordances if len(c)==window]
    return concordances

In [23]:
cgl["conc_lype"] = cgl["lemmata_repl"].apply(lambda x: get_concordances(x, "λύπ*", 31))